<a href="https://colab.research.google.com/github/Saimoguloju/Diet-Recommendation-System-/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [3]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [4]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

For this tutorial, configure the backend for JAX.

In [5]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [6]:
import keras
import keras_nlp

## Load Dataset

In [7]:
!wget -O  diet.txt https://huggingface.co/datasets/shubhamtr/llama2_diet_planner/raw/main/diet.txt

--2025-02-18 03:57:56--  https://huggingface.co/datasets/shubhamtr/llama2_diet_planner/raw/main/diet.txt
Resolving huggingface.co (huggingface.co)... 13.33.45.10, 13.33.45.84, 13.33.45.37, ...
Connecting to huggingface.co (huggingface.co)|13.33.45.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1487007 (1.4M) [text/plain]
Saving to: ‘diet.txt’

diet.txt            100%[===================>]   1.42M  6.40MB/s    in 0.2s    

2025-02-18 03:57:56 (6.40 MB/s) - ‘diet.txt’ saved [1487007/1487007]



Preprocess the data. This tutorial uses a subset of 1000 training examples to execute the notebook faster. Consider using more training data for higher quality fine-tuning.

In [8]:
import re

data = []
with open("diet.txt", encoding="utf-8") as file:
    content = file.read()

# Extracting each instruction-response pair using regex
matches = re.findall(r"\[INST\](.*?)\[/INST\](.*?)(?=<s>|$)", content, re.DOTALL)

for instruction, response in matches:
    instruction = instruction.strip()
    response = response.strip()

    template = "Instruction:\n{}\n\nResponse:\n{}"
    data.append(template.format(instruction, response))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

# Example: Print the first formatted entry
print(data[0])


Instruction:
Sex: Male, Age: 31, Height: 1.77, Weight: 52.5, Hypertension: Yes, Diabetes: No, BMI: 16.76, Level: Underweight, Fitness Goal: Weight Gain, Fitness Type: Muscular Fitness, Diet: Vegetables: (Tomatoes, Garlic, leafy greens, broccoli, carrots, and bell peppers); Protein Intake: (poultry, fish, tofu, legumes, and low-fat dairy products); Juice: (Apple juice, beetroot juice and mango juice)

Response:
Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. I'm not a medical professional, but I can give you general advice that you should talk to your healthcare provider before making any changes to your exercise or diet, especially if you have hypertension. It is important to work with a health care professional to create an individualized plan based on your unique health needs. Always consult a health care professional or dietician for personalized advice based on your specific medical condition, as high blood pressure requires careful m

## Load Model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/). In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

In [9]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [10]:
import keras_nlp

# Define the template correctly with named placeholders
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Define the input prompt
prompt = template.format(
    instruction="Sex: Male, Age: 31, Height: 1.77, Weight: 52.5, Hypertension: Yes, Diabetes: No, BMI: 16.76, Level: Underweight, Fitness Goal: Weight Gain",
    response=""
)

# Initialize Keras NLP Sampler
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)

# Ensure gemma_lm is properly initialized before compilation
# (Make sure you have a valid model instance assigned to `gemma_lm`)
gemma_lm.compile(sampler=sampler)

# Generate text
output = gemma_lm.generate(prompt, max_length=256)
print(output)


Instruction:
Sex: Male, Age: 31, Height: 1.77, Weight: 52.5, Hypertension: Yes, Diabetes: No, BMI: 16.76, Level: Underweight, Fitness Goal: Weight Gain

Response:
1. How often does he exercise?
2. How many days of the week does he exercise?
3. How long does he exercise? (Minutes per day)
4. When does he exercise?

1. 2-3 times/week
2. 6 days/week
3. 45 minutes/session
4. Morning and evening

1. How often does he consume alcohol?
2. When does he consume alcohol?
3. How much does he consume per week?

1. 1 time/week
2. Friday evening
3. About 3 drinks per session

1. What is his typical meal plan during the week?
  a. Breakfast (8:30 AM)
  b. Lunch (11:30 AM)
  c. Snacks (2:00 PM)
  d. Dinner (6:30 PM)
2. What is


## LoRA Fine-tuning

To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [11]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.6 billion to 2.9 million).

In [12]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 996s 959ms/step - loss: 1.0975 - sparse_categorical_accuracy: 0.7653


In [13]:
import keras_nlp

# Define the template correctly with named placeholders
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Define the input prompt
prompt = template.format(
    instruction="Sex: Male, Age: 31, Height: 1.77, Weight: 52.5, Hypertension: Yes, Diabetes: No, BMI: 16.76, Level: Underweight, Fitness Goal: Weight Gain",
    response=""
)

# Initialize Keras NLP Sampler
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)

# Ensure gemma_lm is properly initialized before compilation
# (Make sure you have a valid model instance assigned to `gemma_lm`)
gemma_lm.compile(sampler=sampler)

# Generate text
output = gemma_lm.generate(prompt, max_length=256)
print(output)


Instruction:
Sex: Male, Age: 31, Height: 1.77, Weight: 52.5, Hypertension: Yes, Diabetes: No, BMI: 16.76, Level: Underweight, Fitness Goal: Weight Gain

Response:
Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and holistic way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a health care professional or registered dietitian before making any significant changes to your exercise or diet plan, especially if you have hypertension. Here are some important tips:- Stay hydrated by drinking enough water throughout the day. Monitor your progress and adjust your diet and exercise routine accordingly. Get adequate sleep to support muscle recovery and overall health. Continue to


In [14]:
import keras_nlp

# Define the template correctly with named placeholders
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Define the input prompt
prompt = template.format(
    instruction="Sex: Male, Age: 47, Height: 1.68, Weight: 72.0, Hypertension: Yes, Diabetes: Yes, BMI: 25.51, Level: Overweight, Fitness Goal: Weight Loss, Fitness Type: Cardio Fitness, Diet: Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice,and Beetroot juice)",
    response=""
)

# Initialize Keras NLP Sampler
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)

# Ensure gemma_lm is properly initialized before compilation
# (Make sure you have a valid model instance assigned to `gemma_lm`)
gemma_lm.compile(sampler=sampler)

# Generate text
output = gemma_lm.generate(prompt, max_length=256)
print(output)


Instruction:
Sex: Male, Age: 47, Height: 1.68, Weight: 72.0, Hypertension: Yes, Diabetes: Yes, BMI: 25.51, Level: Overweight, Fitness Goal: Weight Loss, Fitness Type: Cardio Fitness, Diet: Vegetables: (Garlic, Mushroom, Green Papper, Icebetg Lettuce); Protein Intake: (Baru Nuts, Beech Nuts, Hemp Seeds, Cheese Spandwich); Juice: (Apple Juice, Mango juice,and Beetroot juice)

Response:
Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight loss in a healthy and balanced way, focusing on exercise and nutrition. Keep in mind that weight lossshould be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a healthcare professional or registered dietitian before making any significant changes to your exercise or diet plan. Here are some important tips:- Stay hydrated by drinking enough water throughout the day. Monitor your progress and adjust you